In [1]:
import warnings
warnings.filterwarnings("ignore")
from utils import *
import pickle

def extract_transform(path, keep_cols, cols_rows_with_missing_values, new_binary_target):
    _ , spark_df = SparkDataProcessor().process_data(
        data_csv_path= path,
        )

    return prepare_spark_df(
        spark_df=spark_df,
        keep_cols=keep_cols,
        cols_rows_with_missing_values=cols_rows_with_missing_values,
        new_binary_target = new_binary_target,
    )
# compare_dataframes(df1, df2)
# df1.columns, df2.columns
OPBG = [ 'código','age at diagnosis (months)',            
            'ntbc dosis mg/kg/day',
            'ntbc levels (dbs)',
            'sca (urine)',
            'methionine (plasma)',
            'tyrosine (plasma)',
            'phenylalanine (plasma)',
            'pt (sec)',
            'bili total',
            'gpt',
            'got',
            'ggt',
            'alkaline phosphatase',
            'alfa-fetoprotein','glicemia']


# OPBG = [ 'age at diagnosis (months)',    'código',   
#                 'ntbc dosis mg/kg/day',
#                 'ntbc levels (dbs)',
#                 'sca (urine)',
#                 'methionine dbs',
#                 'tyrosine dbs',
#                 'phenylalanine dbs',
#                 'methionine (plasma)',
#                 'tyrosine (plasma)',
#                 'phenylalanine (plasma)',
#                 'inr',
#                 'pt (sec)',
#                 'bili total',
#                 'gpt',
#                 'got',
#                 'ggt',
#                 'alkaline phosphatase',
#                 'alfa-fetoprotein',]
    
# df1 = pd.read_csv('data/tirosinemia.csv').loc[:,OPBG]
# df2 = pd.read_csv('data/tirosinemia_italia.csv').loc[:,OPBG]

In [3]:
binary_target = 'Alpha-Fet'
df_train, _ = extract_transform("data/tirosinemia.csv",                 OPBG, ['alfa-fetoprotein', 'sca (urine)'],binary_target)
df_train.drop('código', axis=1, inplace=True)
df_test, df_test_raw  = extract_transform("data/tirosinemia_italia.csv", OPBG, ['alfa-fetoprotein', 'sca (urine)'],binary_target)
Rome_mask = df_test_raw['código'].str.startswith('R')

df_test_FLOR  = encode_and_convert (df_test[~Rome_mask].drop('código', axis=1).reset_index(inplace=False, drop=True))
df_test_ROME  = encode_and_convert (df_test[Rome_mask].drop('código', axis=1).reset_index(inplace=False, drop=True))
df_test_FLOR['Alpha-Fet'] = df_test_FLOR['Alpha-Fet'].astype('int8').astype('category')
df_test_ROME['Alpha-Fet'] = df_test_ROME['Alpha-Fet'].astype('int8').astype('category')


compare_dataframes(df_test_ROME, df_test_FLOR) == compare_dataframes(df_train, df_test_ROME)



Both CSV files have the same number of columns: 15
Column names are the same.
Column order is the same.
All common columns have the same data types.
Both CSV files have the same number of columns: 15
Column names are the same.
Column order is the same.
All common columns have the same data types.


True

In [5]:
df_train.to_parquet("data/train.parquet.gzip")
df_test_ROME.to_parquet("data/test.parquet.gzip")

In [6]:
ray.shutdown()
ray_df_train = ray.put(df_train)
ray_df_test  = ray.put(df_test_ROME)

2023-06-07 20:02:14,803	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [7]:
features : list[str] = df_train.columns.tolist()
features.remove(binary_target)
features

['age at diagnosis (months)',
 'ntbc dosis mg/kg/day',
 'ntbc levels (dbs)',
 'sca (urine)',
 'methionine (plasma)',
 'tyrosine (plasma)',
 'phenylalanine (plasma)',
 'pt (sec)',
 'bili total',
 'gpt',
 'got',
 'ggt',
 'alkaline phosphatase',
 'glicemia']

In [9]:
%%time

studies = make_multiple_studies(
    ray_df_train, 
    features = features,
    targets = [binary_target],
    n_trials=5,
    Independent_testset = False,
    Independent_testset_df = ray.put(df_test_ROME)
)

studies = ray.get(studies)

(make_a_study pid=8462) [I 2023-06-07 20:38:05,412] A new study created in memory with name: ntbc dosis mg/kg/day (Alpha-Fet)
(make_a_study pid=8459) [I 2023-06-07 20:02:38,457] Trial 3 finished with values: [0.8888888888888888, 0.15080341696739197] and parameters: {'seed': 9771, 'max_depth': 7, 'eta': 0.18521273464702218, 'subsample': 0.21338326784327474, 'lambda': 0.8260651652673303, 'alpha': 0.6849885136760645, 'scale_pos_weight': 4.489663040047078, 'kfold_splits': 5}.  [repeated 3x across cluster]
(make_a_study pid=8459) [I 2023-06-07 20:38:05,412] A new study created in memory with name: age at diagnosis (months) (Alpha-Fet)
(make_a_study pid=8460) [I 2023-06-07 20:38:05,412] A new study created in memory with name: ntbc levels (dbs) (Alpha-Fet)
(make_a_study pid=8452) [I 2023-06-07 20:38:05,412] A new study created in memory with name: sca (urine) (Alpha-Fet)
(make_a_study pid=8452) [I 2023-06-07 20:38:10,504] Trial 3 finished with values: [0.6818181818181818, 0.0314754918217659]

CPU times: user 784 ms, sys: 729 ms, total: 1.51 s
Wall time: 34 s


: 

In [ ]:
with open('chile.pickle', 'wb') as handle:
    pickle.dump(studies, handle, protocol=pickle.HIGHEST_PROTOCOL)    